In [ ]:
# module import
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.tri as tri
import cartopy.crs as ccrs
import pyproj
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.path as mpath
import seaborn as sns
import datetime
import cftime
from matplotlib import gridspec
import cartopy.feature as cft
import pandas as pd
import string
import cmocean as cm
import math
import proplot as pplt
import xesmf as xe
import netCDF4 as nc
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.image as mpimg

## GENERAL DATA

In [ ]:
## DATA
# Mask with basins
netCDF_basins_file = '/Users/jcaillet/Documents/Elmer/DATA/BASINS/schmidtko_initmip8km_means.nc'
# Mask with iceshelves
netCDF_shelf_file = '/Users/jcaillet/Documents/Elmer/DATA/BASINS/Mask_Iceshelf_IMBIE.nc'
# Mask with melt per basin (literature)
netCDF_melt_file = '/Users/jcaillet/Documents/Elmer/DATA/MELT/Melt_Rignot_Adusumilli_large_basin.nc'
# Mask with melt per iceshelf (literature)
netCDF_melt_file1 = '/Users/jcaillet/Documents/Elmer/DATA/MELT/Melt_Rignot_Adusumilli.nc'

# read data
Basin = xr.open_mfdataset(netCDF_basins_file)
Iceshelf = xr.open_mfdataset(netCDF_shelf_file)
Melt_basin = xr.open_mfdataset(netCDF_melt_file)
Melt_iceshelf = xr.open_mfdataset(netCDF_melt_file1)

## DATA FROM ELMER

In [ ]:
# with correction
netCDF_res_file = '/Users/jcaillet/Documents/Elmer/ISMIP6/Calage_fonte/ismip6_fluxes_ismip6_w1-histae_001.nc'
netCDF_res_file1 = '/Users/jcaillet/Documents/Elmer/ISMIP6/Calage_fonte/ismip6_states_ismip6_w1-histae_001.nc'

# without correction
netCDF_res_file2 = '/Users/jcaillet/Documents/Elmer/ISMIP6/Calage_fonte/ismip6_fluxes_ant50.gl1-ismip6_ronja_1.nc'
netCDF_res_file3 = '/Users/jcaillet/Documents/Elmer/ISMIP6/Calage_fonte/ismip6_states_ant50.gl1-ismip6_ronja_1.nc'

Result = xr.open_mfdataset(netCDF_res_file)
Mask = xr.open_mfdataset(netCDF_res_file1)
Result2 = xr.open_mfdataset(netCDF_res_file2)
Mask2 = xr.open_mfdataset(netCDF_res_file3)

Melt_iceshelf['Nisf'] = Melt_iceshelf['Nisf'] + 1
Melt_iceshelf = Melt_iceshelf.rename({'Nisf':'basins_isf'})
Melt_iceshelf['basins_isf'] = Melt_iceshelf['basins_isf'] + 0.0

# fonte basins Ronja and conversion kg/s in Gt/year
Meltb = (Result.libmassbffl[0,:] * Result.cell_area[0,:] * Mask.sftflf[0,:]).groupby(Result.basins[0,:]).sum()
Meltb_final = - Meltb / 1000000000000 * 31536000

# fonte iceshelf
Melti = (Result.libmassbffl[0,:] * Result.cell_area[0,:] * Mask.sftflf[0,:]).groupby(Result.basins_isf[0,:]).sum()
Melti_final = - Melti / 1000000000000 * 31536000

Melti2 = (Result2.libmassbffl[0,:] * Result2.cell_area[0,:] * Mask2.sftflf[0,:]).groupby(Result2.basins_isf[0,:]).sum()
Melti_final2 = - Melti2 / 1000000000000 * 31536000

Melti_final_select = Melti_final.sel(basins_isf = Melt_iceshelf['basins_isf'])
Melti_final_select2 = Melti_final2.sel(basins_isf = Melt_iceshelf['basins_isf'])

## FIGURE

In [ ]:
# figure per iceshelf
sns.set_context('paper')

B = np.array(Melt_iceshelf.name_isf)
D = np.arange(0,66,1)
DM = np.arange(-0.25,65,1)
DP = np.arange(0.25,65.3,1)
fig2 = plt.figure(figsize = (12, 6))
ax = plt.subplot(111)
plt.errorbar(D, Melt_iceshelf.melt_Rignot, yerr = Melt_iceshelf['melt_uncertainty_Rignot'], color = 'orange', fmt = 'o', ms = 2, elinewidth = 0.75, capsize = 1, label = 'Rignot: 2003-2008 estimates', zorder=1)
plt.errorbar(DP, Melt_iceshelf.melt_Adusumilli, yerr = Melt_iceshelf['melt_uncertainty_Adusumilli'], color = 'firebrick', fmt = 'o', ms = 2, elinewidth = 0.75, capsize = 1, label = 'Adusumilli: 1994-2018 estimates', zorder=2)
plt.errorbar(DM, Melti_final_select2, yerr = 0,  color = 'lightblue', fmt = 'o', ms = 2, elinewidth = 0, capsize = 0, label = 'Model before calibration', zorder=2)
plt.errorbar(DM, Melti_final_select, yerr = 0, color = 'dodgerblue', fmt = 'o', ms = 2, elinewidth = 0, capsize = 0, label = 'Model after calibration', zorder=3)
plt.annotate("", xy=(-0.5, -50), xytext=(6.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(6.5, -50), xytext=(11.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(11.5, -50), xytext=(12.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(12.5, -50), xytext=(17.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(17.5, -50), xytext=(21.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(21.5, -50), xytext=(25.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(25.5, -50), xytext=(31.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(31.5, -50), xytext=(32.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(32.5, -50), xytext=(36.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(36.5, -50), xytext=(39.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(39.5, -50), xytext=(44.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(44.5, -50), xytext=(47.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(47.5, -50), xytext=(51.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(51.5, -50), xytext=(54.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(54.5, -50), xytext=(58.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(58.5, -50), xytext=(59.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(59.5, -50), xytext=(60.5, -50), arrowprops=dict(arrowstyle="<->"))
plt.annotate("", xy=(60.5, -50), xytext=(65.5, -50), arrowprops=dict(arrowstyle="<->"))

plt.text(2.75, -70, "1", fontsize=10)
plt.text(8.75, -70, "2", fontsize=10)
plt.text(11.75, -70, "3", fontsize=10)
plt.text(14.75, -70, "4", fontsize=10)
plt.text(19.25, -70, "5", fontsize=10)
plt.text(23.25, -70, "6", fontsize=10)
plt.text(28.25, -70, "7", fontsize=10)
plt.text(31.75, -70, "8", fontsize=10)
plt.text(34.25, -70, "9", fontsize=10)
plt.text(37.5, -70, "10", fontsize=10)
plt.text(41.5, -70, "11", fontsize=10)
plt.text(45.5, -70, "12", fontsize=10)
plt.text(49.0, -70, "13", fontsize=10)
plt.text(52.5, -70, "14", fontsize=10)
plt.text(56.0, -70, "15", fontsize=10)
plt.text(58.25, -70, "16", fontsize=10)
plt.text(59.5, -70, "17", fontsize=10)
plt.text(62.5, -70, "18", fontsize=10)


plt.legend(loc='upper left', prop={"size":10})
plt.ylabel('Melt (Gt yr$^{-1}$)', fontsize = 10)
plt.xticks(D, B)
plt.ylim(-75,200)
plt.xlim(-0.75,65.75)
plt.xticks(fontsize=10, rotation=90)
plt.hlines(0, -1, 66, linewidth = 0.5, linestyle='--',colors= 'black')
for k in np.arange(0,65.8,2):
    plt.axvspan(k-0.5, k+0.5, alpha=0.1, color='grey')
    
# image in the right corner
Ant = mpimg.imread('/Users/jcaillet/Documents/Elmer/CMIP6/FIGURES/schema2.png')
imagebox = OffsetImage(Ant, zoom=0.17)
ab = AnnotationBbox(imagebox, (63, 170), zorder=10)
ax.add_artist(ab)

plt.tight_layout(rect=[0.0, 0.0, 0.90, 0.80])    

fig2.savefig('/Users/jcaillet/Documents/Elmer/ISMIP6/Calage_fonte/fig_comp_iceshelf.pdf')